# Traveling Salesman Problem

## Model Initialization

In [2]:
# import Glop linear solver package
from ortools.linear_solver import pywraplp as glp
import csv
# initialize model object
mymodel = glp.Solver('Traveling Salesman', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

## Parameters

In [3]:
# import cost matrix as data frame
#cost = [[ 0.0,  5.0, 15.0, 10.0], # cost from node 0
#        [ 5.0,  0.0, 12.0, 20.0],  # node 1
#        [15.0, 12.0,  0.0,  6.0],  # node 2
#        [10.0, 20.0,  6.0,  0.0]]  # node 3

# coerce data type = integer / numeric?
# Probably need a way to create the dummny data from a regular cost matrix

# row = starting node, column = ending node (i to j)

In [4]:
# import csv distance matrix as list
with open('Orders_Data_TSP.csv', newline='') as f:
    reader = csv.reader(f)
    cost = list(reader)

# remove blanks & converting to integers
for i in range(len(cost)):
    cost[i] = [float(item) for item in cost[i] if item]
cost = [item for item in cost if item]

In [5]:
cost

[[0.0, 100.0, 81.0, 99.0, 61.0, 111.0, 16.0, 76.0, 67.0, 84.0, 102.0],
 [100.0, 0.0, 97.0, 98.0, 41.0, 93.0, 97.0, 92.0, 48.0, 51.0, 9.0],
 [81.0, 97.0, 0.0, 19.0, 66.0, 36.0, 66.0, 6.0, 51.0, 47.0, 92.0],
 [99.0, 98.0, 19.0, 0.0, 74.0, 18.0, 84.0, 23.0, 58.0, 47.0, 92.0],
 [61.0, 41.0, 66.0, 74.0, 0.0, 77.0, 56.0, 61.0, 17.0, 34.0, 42.0],
 [111.0, 93.0, 36.0, 18.0, 77.0, 0.0, 97.0, 38.0, 60.0, 45.0, 86.0],
 [16.0, 97.0, 66.0, 84.0, 56.0, 97.0, 0.0, 60.0, 59.0, 73.0, 98.0],
 [76.0, 92.0, 6.0, 23.0, 61.0, 38.0, 60.0, 0.0, 46.0, 43.0, 88.0],
 [67.0, 48.0, 51.0, 58.0, 17.0, 60.0, 59.0, 46.0, 0.0, 18.0, 45.0],
 [84.0, 51.0, 47.0, 47.0, 34.0, 45.0, 73.0, 43.0, 18.0, 0.0, 45.0],
 [102.0, 9.0, 92.0, 92.0, 42.0, 86.0, 98.0, 88.0, 45.0, 45.0, 0.0]]

In [6]:
# model parameters
N = len(cost) # number of nodes in data frame (including the dummy node)
M = N + 1000.0 # arbitrarily large number

In [7]:
N

11

In [8]:
M

1011.0

## Decision Variables

In [9]:
use_arc = [list(range(1 + N * i, 1 + N * (i+1))) for i in range(N)] 
for i in range(N):
    for j in range(N):
        use_arc[i][j] = mymodel.IntVar(0, 1, str(i) + "." + str(j))
use_arc

[[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.10],
 [1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 1.10],
 [2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 2.10],
 [3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 3.10],
 [4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 4.10],
 [5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 5.10],
 [6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 6.10],
 [7.0, 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 7.10],
 [8.0, 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 8.10],
 [9.0, 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 9.10],
 [10.0, 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 10.10]]

In [10]:
# create position variable (lambda / u)
pos = list(range(N))
pos[0] = mymodel.IntVar(0, 0, 'p_0')
for i in range(1,N):
    pos[i] = mymodel.IntVar(1, N-1, "p_" + str(i))
pos

[p_0, p_1, p_2, p_3, p_4, p_5, p_6, p_7, p_8, p_9, p_10]

## Objective

In [11]:
# create objective function
shortest_route = mymodel.Objective()
shortest_route.SetMinimization()
for i in range(N):
    for j in range(N):
        shortest_route.SetCoefficient(use_arc[i][j], cost[i][j])

## Constraints

### Arc constraints

In [12]:
# Balance - Out
# 2.1 + 2.2 + 2.3 + 2.4 + 2.5 = 1

# Balance - In
# 1.2 + 2.2 + 3.2 + 4.2 + 5.2 = 1

# Flow Out

Flow_out = list(range(N))

for i in range(N): 
    Flow_out[i] = mymodel.Constraint(1, 1)
    for j in range(N):
        if i != j: Flow_out[i].SetCoefficient(use_arc[i][j], 1)
        
# Flow In

Flow_in = list(range(N))

for i in range(N):
    Flow_in[i] = mymodel.Constraint(1, 1)
    for j in range(N): 
        if i!= j: Flow_in[i].SetCoefficient(use_arc[j][i], 1)

### Position Constraints

In [13]:
# position 1 = 1
#position_1 = mymodel.Constraint(1, 1)
#position_1.SetCoefficient(pos[0], 1)

# position N = N
#position_N = mymodel.Constraint(1, 1)
#position_N.SetCoefficient(pos[N-1], N)

# MTZ Constraint

# u[i] - u[j] + (n-1)*x[i][j] <= (n-2)

MTZ = [list(range(1 + N * i, 1 + N * (i+1))) for i in range(N)] 

for i in range(1,N):
    for j in range(1,N):
        MTZ[i][j] = mymodel.Constraint(-mymodel.infinity(), N-2)
        MTZ[i][j].SetCoefficient(use_arc[i][j], N-1)
        MTZ[i][j].SetCoefficient(pos[i], 1)
        MTZ[i][j].SetCoefficient(pos[j], -1)

In [14]:
# Solve the model and print optimal solution
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Optimal Value = %.2f' % shortest_route.Value())

# Display optimal solution
for i in range(N):
    print('pos[%d] = %d' % (i, pos[i].solution_value()))
    for j in range(N):
        print('From ', i, ' to ', j, ': ', use_arc[i][j].solution_value(), sep = '')
        
#        if i == N-1 or i == j:
#            continue
#        elif j == N-1:
 #           j = 1
 #           print('From node ', i + 1, ' to node ', j + 1, ': ', use_arc[j][i].solution_value(), sep = '')
 #       else:
  #          print('From node ', i + 1, ' to node ', j + 1, ': ', use_arc[j][i].solution_value(), sep = '') 

Solution Status = 0
Number of variables = 132
Number of constraints = 122
Optimal Solution:
Optimal Value = 338.00
pos[0] = 0
From 0 to 0: 0.0
From 0 to 1: 0.0
From 0 to 2: 0.0
From 0 to 3: 0.0
From 0 to 4: 0.0
From 0 to 5: 0.0
From 0 to 6: 1.0
From 0 to 7: 0.0
From 0 to 8: 0.0
From 0 to 9: 0.0
From 0 to 10: 0.0
pos[1] = 9
From 1 to 0: 0.0
From 1 to 1: 0.0
From 1 to 2: 0.0
From 1 to 3: 0.0
From 1 to 4: 1.0
From 1 to 5: 0.0
From 1 to 6: 0.0
From 1 to 7: 0.0
From 1 to 8: 0.0
From 1 to 9: 0.0
From 1 to 10: 0.0
pos[2] = 3
From 2 to 0: 0.0
From 2 to 1: 0.0
From 2 to 2: 0.0
From 2 to 3: 1.0
From 2 to 4: 0.0
From 2 to 5: 0.0
From 2 to 6: 0.0
From 2 to 7: 0.0
From 2 to 8: 0.0
From 2 to 9: 0.0
From 2 to 10: 0.0
pos[3] = 4
From 3 to 0: 0.0
From 3 to 1: 0.0
From 3 to 2: 0.0
From 3 to 3: 0.0
From 3 to 4: 0.0
From 3 to 5: 1.0
From 3 to 6: 0.0
From 3 to 7: 0.0
From 3 to 8: 0.0
From 3 to 9: 0.0
From 3 to 10: 0.0
pos[4] = 10
From 4 to 0: 1.0
From 4 to 1: 0.0
From 4 to 2: 0.0
From 4 to 3: 0.0
From 4 to

## Plot the Solution

In [15]:
import matplotlib as plt
import networkx as nx

In [16]:
# import csv x, y coord matrix as list
with open('XYcoord.csv', newline='') as f:
    reader = csv.reader(f)
    coord = list(reader)

# remove blanks & converting to integers
for i in range(len(coord)):
    coord[i] = [float(item) for item in coord[i] if item]
coord = [item for item in coord if item]
coord

[[0.0, 0.0],
 [13.0, 99.0],
 [77.0, 26.0],
 [91.0, 39.0],
 [19.0, 58.0],
 [96.0, 56.0],
 [16.0, 2.0],
 [71.0, 27.0],
 [36.0, 57.0],
 [52.0, 66.0],
 [22.0, 100.0]]

In [17]:
x = []
y = []
for i in range(len(coord)):
    x.append(coord[i][0])
    y.append(coord[i][1])
y

[0.0, 99.0, 26.0, 39.0, 58.0, 56.0, 2.0, 27.0, 57.0, 66.0, 100.0]

In [18]:
location = []
node = []
edge = []

for i in range(N):
    location.append(int(pos[i].solution_value()))
    node.append(int(i))
    for j in range(N):
        if use_arc[i][j].solution_value() == 1.0:
            edge.append([i, j, cost[i][j]])
edge

[[0, 6, 16.0],
 [1, 4, 41.0],
 [2, 3, 19.0],
 [3, 5, 18.0],
 [4, 0, 61.0],
 [5, 9, 45.0],
 [6, 7, 60.0],
 [7, 2, 6.0],
 [8, 10, 45.0],
 [9, 8, 18.0],
 [10, 1, 9.0]]

In [19]:
G = nx.Graph()

In [24]:

for i in range(len(edge)):
    G.add_edge(edge[i][0], edge[i][1], edge[i][2])

TypeError: add_edge() takes 3 positional arguments but 4 were given

In [26]:
range(len(edge))

range(0, 11)

In [ ]:
nodelist = []
for i in range(N):
    nodelist.append([i, x[i], y[i]])
nodelist

In [ ]:
for i in range(len(node)):
    node_positions = {node[i] : (x[i], -y[i])}
node_positions

In [ ]:
for i, elrow in edgelist.iterrows():
    g.add_edge(elrow[0], elrow[1], attr_dict=elrow[2:].to_dict())

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(g, pos=node_positions, edge_color='blue', node_size=10, node_color='black')
plt.title(print('TSP Solution with %d nodes' % N), size=15)
plt.show()

In [ ]:
print('pos[%d] = %d' % (i, pos[i].solution_value()))